In [ ]:
# read the json file
import json
with open('final_data.json') as file:
    intents = json.load(file)

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
# import the requiremnets
import pickle
vect = pickle.load(open("vectoriser.pkl","rb"))
tags = pickle.load(open("tag.pkl","rb"))
x_final = pickle.load(open("x_final.pkl","rb"))

In [ ]:
# load the Chatbot model
import tensorflow as tf
model = tf.keras.models.load_model('Ch_model.h5')

In [ ]:
# import required libraries
import re
import nltk
import random
from textblob import TextBlob
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

# import stemmer to conver the words to its base form
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# create vectorize data
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer().fit(x_final)
print(vect)

TfidfVectorizer()


In [ ]:
# define the function to check the spellings
def spell_checker(sentence):

  # this list is for words which is to be ingnored
  ignore_words = []

  # open the file and load the ingnore words
  try:
    file = open("words.txt",'r')
    for word in file.readlines():
      ignore_words.append(word[:-1])
  except FileNotFoundError:
    print("Creating new word.txt file....")

  # tokenize the sentence and corret it
  token = nltk.word_tokenize(sentence)
  for word in token: 
    corrected = TextBlob(word)
    corrected = str(corrected.correct())

    # if word is incorrect then give warring to user
    if word not in ignore_words:
      if word != corrected:
        print(f"Warning: {word} word looks incorrect")
        print("sugetion: ",corrected)
        print("")
        a = input("if you found this suggeion helpful please enter Y ans N if not: ")

        # if word is is correct then store it in file
        if a=='N':
          ignore_words.append(word)
          file = open('words.txt','a+')
          file.write(str(word)+"\n")
          file.close()
          return sentence
    

        # if word is incorrect then ask user to enter question again
        elif a=="Y":
          sentence = input("try to enter question again\n")
          return responce(sentence)

  return sentence

In [ ]:
# define the function to input the sentence and preprocess it
def input_sentence(sentence):

  if sentence != "":
    sentence = spell_checker(sentence)

  # removing any abnormal words in sentence
  clean_sentece = re.sub('[^a-zA-Z\s\w]','',sentence)

  # processing the sentence
  sentence_words = nltk.word_tokenize(clean_sentece)
  processed_sentence = []
  for w in sentence_words:
    if w not in stopwords:
      word = stemmer.stem(w.lower())
      processed_sentence.append(word)
  processed_sentence = ' '.join(processed_sentence)

  print(processed_sentence)
  return processed_sentence

In [ ]:
# defining the function to indentify the intent of the input
def identify_intent(sentence):
  
  # predict the results using ANN model
  result = model.predict(vect.transform([input_sentence(sentence)]).toarray())

  # printing the max score
  max_score = max(result[0])
  print(max_score)

  # if the score is more than thresh hold then process it future
  if max_score >= 0.25:
    index = 0

    # getting the index of the intent to get the appropriate responce
    for score in result:
      for prob in score:
        index +=1
        if prob == max_score:
          break
    return index
  
  # if the score is less than threshold then ask user new question
  elif max_score <= 0.25:
    print("""Soory but i don't have knowledge about it. I will let you know i about this is some time""");

    # store the question to file to prvide knowledge avout it later
    file = open('questions.txt','a+')
    file.write(str(sentence)+"\n")
    file.close()

    # ask the user new question
    return identify_intent(input("Please try another question: "))

In [ ]:
# define the function to fetch the responce
def responce(sentence):

  # detect the tag using its index to fetch the proper respnce
  output_tag = identify_intent(sentence)
  print("hi",output_tag)
  result = tags[output_tag-1]

  # select appropriate responce and show it
  for intent in intents['Intents']:
    if intent['tag'] == result:
      print(intent['tag'])
      responce = random.choice(intent['responses'])
      break
  return responce

In [ ]:
a = "a"
print("please 'enter' to exit form the conversation")
while a!='':
  a = input("question: ")
  if a=='':
    break
  output = responce(a)
  print("chatbot:", output)
  print("----------------------------------------------------------------------------------------------------------------")

please 'enter' to exit form the conversation
sugetion:  copied

coviad
0.40747043
hi 567
thanks
chatbot: My pleasure
----------------------------------------------------------------------------------------------------------------


In [ ]:
# use of LSTM model
# voice input output